<a href="https://colab.research.google.com/github/adwitaarora/ai-hw-lab/blob/main/try2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. COLLECTING DATA


In [1]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [2]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import shutil
import sys 

In [ ]:
pip install datasets

In [4]:
from datasets import load_dataset

In [5]:
dataset = load_dataset('sem_eval_2018_task_1','subtask5.english')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/6838 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3259 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/886 [00:00<?, ? examples/s]

Dataset sem_eval_2018_task_1 downloaded and prepared to /root/.cache/huggingface/datasets/sem_eval_2018_task_1/subtask5.english/1.1.0/a7c0de8b805f1988b118882fb289ccfbbeb9085c7820b6f046b5887e234af182. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
dataset['train'][0]

{'ID': '2017-En-21441',
 'Tweet': "“Worry is a down payment on a problem you may never have'. \xa0Joyce Meyer.  #motivation #leadership #worry",
 'anger': False,
 'anticipation': True,
 'disgust': False,
 'fear': False,
 'joy': False,
 'love': False,
 'optimism': True,
 'pessimism': False,
 'sadness': False,
 'surprise': False,
 'trust': True}

In [8]:
train_df=dataset['train'].to_pandas()
train2_df=dataset['validation'].to_pandas()
test_df=dataset['test'].to_pandas()

In [9]:
print(len(train_df))
print(len(train2_df))

6838
886


In [10]:
train_df=pd.concat([train_df,train2_df])

In [11]:
train_df.drop(labels=['ID'],axis=1,inplace=True)


In [12]:
train_df.columns

Index(['Tweet', 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'love',
       'optimism', 'pessimism', 'sadness', 'surprise', 'trust'],
      dtype='object')

In [13]:
train_df.head()

,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,“Worry is a down payment on a problem you may ...,False,True,False,False,False,False,True,False,False,False,True
1,Whatever you decide to do make sure it makes y...,False,False,False,False,True,True,True,False,False,False,False
2,@Max_Kellerman it also helps that the majorit...,True,False,True,False,True,False,True,False,False,False,False
3,Accept the challenges so that you can literall...,False,False,False,False,True,False,True,False,False,False,False
4,My roommate: it's okay that we can't spell bec...,True,False,True,False,False,False,False,False,False,False,False


In [14]:
target_list = ['anger',	'anticipation',	'disgust','fear',
	'joy',	'love',	'optimism',	'pessimism',	'sadness',
  	'surprise',	'trust']

In [15]:
train_df.anger = train_df.anger.replace({True: 1, False: 0})
train_df.anticipation = train_df.anticipation.replace({True: 1, False: 0})
train_df.disgust = train_df.disgust.replace({True: 1, False: 0})
train_df.joy = train_df.joy.replace({True: 1, False: 0})
train_df.fear = train_df.fear.replace({True: 1, False: 0})
train_df.love = train_df.love.replace({True: 1, False: 0})
train_df.optimism = train_df.optimism.replace({True: 1, False: 0})
train_df.pessimism = train_df.pessimism.replace({True: 1, False: 0})
train_df.sadness = train_df.sadness.replace({True: 1, False: 0})
train_df.surprise = train_df.surprise.replace({True: 1, False: 0})
train_df.trust = train_df.trust.replace({True: 1, False: 0})

In [16]:
train_df.head()

,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,“Worry is a down payment on a problem you may ...,0,1,0,0,0,0,1,0,0,0,1
1,Whatever you decide to do make sure it makes y...,0,0,0,0,1,1,1,0,0,0,0
2,@Max_Kellerman it also helps that the majorit...,1,0,1,0,1,0,1,0,0,0,0
3,Accept the challenges so that you can literall...,0,0,0,0,1,0,1,0,0,0,0
4,My roommate: it's okay that we can't spell bec...,1,0,1,0,0,0,0,0,0,0,0


# 2. Training the Model


In [17]:
#hyperparameters
MAX_LEN = 256
TRAIN_BATCH_SIZE = 15
VALID_BATCH_SIZE = 15
EPOCHS = 2
LEARNING_RATE = 1e-05

In [18]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 96.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.2 MB/s eta 0:00:00


In [19]:
from transformers import BertTokenizer, BertModel

In [20]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [21]:
class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.title = df['Tweet']
        self.targets = self.df[target_list].values
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index])
        }

In [22]:
train_size = 0.8
train_df = train_df.sample(frac=train_size, random_state=200).reset_index(drop=True)
val_df = train_df.drop(train_df.index).reset_index(drop=True)

In [23]:
train_dataset = CustomDataset(train_df, tokenizer, MAX_LEN)
valid_dataset = CustomDataset(val_df, tokenizer, MAX_LEN) 

In [24]:
train_data_loader = torch.utils.data.DataLoader(train_dataset, 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

val_data_loader = torch.utils.data.DataLoader(valid_dataset, 
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

In [25]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [26]:
device

device(type='cuda')

In [ ]:
pip install torchmetrics

In [29]:
from torchmetrics.classification import MultilabelAccuracy

In [30]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into       
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss 
    return model, optimizer, checkpoint['epoch'], valid_loss_min.item()

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

In [ ]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert_model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, 11)
    
    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids, 
            attention_mask=attn_mask, 
            token_type_ids=token_type_ids
        )
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output

model = BERTClass()
model.to(device)

In [32]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [47]:
def acc_fn(outputs,targets):
  metric = MultilabelAccuracy(num_labels=11)
  acc=metric(outputs,targets)
  print(acc)
  return acc


In [33]:
val_targets=[]
val_outputs=[]

In [45]:
def train_model(n_epochs, training_loader, validation_loader, model, 
                optimizer, checkpoint_path, best_model_path):
   
  # initialize tracker for minimum validation loss
  valid_loss_min = np.Inf
   
 
  for epoch in range(1, n_epochs+1):
    train_loss = 0
    valid_loss = 0

    model.train()
    print('############# Epoch {}: Training Start   #############'.format(epoch))
    for batch_idx, data in enumerate(training_loader):
        #print('yyy epoch', batch_idx)
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)
        val_targets=targets
        val_outputs=outputs 
        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        

        
        #if batch_idx%5000==0:
         #   print(f'Epoch: {epoch}, Training Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #print('before loss data in training', loss.item(), train_loss)
        train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))
        #print('after loss data in training', loss.item(), train_loss)
    
    print('############# Epoch {}: Training End     #############'.format(epoch))
    
    print('############# Epoch {}: Validation Start   #############'.format(epoch))
    ######################    
    # validate the model #
    ######################
 
    model.eval()
   
    with torch.no_grad():
      for batch_idx, data in enumerate(validation_loader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)

            loss = loss_fn(outputs, targets)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))
            val_targets.extend(targets.cpu().detach().numpy().tolist())
            val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

      print('############# Epoch {}: Validation End     #############'.format(epoch))
      # calculate average losses
      #print('before cal avg train loss', train_loss)
      if len(training_loader)!=0:
        train_loss = train_loss/len(training_loader)
      if len(validation_loader)!=0:
        valid_loss = valid_loss/len(validation_loader)
      # print training/validation statistics 
      print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
      
      # create checkpoint variable and add important data
      checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
      }
       
        # save checkpoint
      # save_ckp(checkpoint, False, checkpoint_path, best_model_path)
        
      ## TODO: save the model if validation loss has decreased
      # if valid_loss <= valid_loss_min:
      #   print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
      #   # save checkpoint as best model
      #   save_ckp(checkpoint, True, checkpoint_path, best_model_path)
      #   valid_loss_min = valid_loss

    print('############# Epoch {}  Done   #############\n'.format(epoch))

  return model

In [35]:
checkpoint_path = "/content/drive/MyDrive/datasets/multi-label/curr_ckpt" 
best_model_path = "/content/drive/MyDrive/datasets/multi-label/best_model.pt"

In [46]:
trained_model = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, checkpoint_path, best_model_path)
     

############# Epoch 1: Training Start   #############
############# Epoch 1: Training End     #############
############# Epoch 1: Validation Start   #############
############# Epoch 1: Validation End     #############
Epoch: 1 	Avgerage Training Loss: 0.000701 	Average Validation Loss: 0.000000
############# Epoch 1  Done   #############

############# Epoch 2: Training Start   #############
############# Epoch 2: Training End     #############
############# Epoch 2: Validation Start   #############
############# Epoch 2: Validation End     #############
Epoch: 2 	Avgerage Training Loss: 0.000630 	Average Validation Loss: 0.000000
############# Epoch 2  Done   #############



In [49]:
len(val_outputs)

0

In [ ]:
test_df

,ID,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,2018-En-01559,@Adnan__786__ @AsYouNotWish Dont worry Indian ...,True,True,False,False,False,False,True,False,False,False,True
1,2018-En-03739,"Academy of Sciences, eschews the normally sobe...",False,False,True,False,False,False,False,False,False,False,False
2,2018-En-00385,I blew that opportunity -__- #mad,True,False,True,False,False,False,False,False,True,False,False
3,2018-En-03001,This time in 2 weeks I will be 30... 😥,False,False,False,False,True,False,False,False,True,False,False
4,2018-En-01988,#Deppression is real. Partners w/ #depressed p...,False,False,False,True,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3254,2018-En-03848,shaft abrasions from panties merely shifted to...,True,False,False,False,False,False,False,True,False,False,False
3255,2018-En-00416,@lomadia heard of Remothered? Indie horror gam...,False,True,False,False,False,False,False,True,False,False,False
3256,2018-En-03717,All this fake outrage. Y'all need to stop 🤣,True,False,True,False,False,False,False,False,False,False,False
3257,2018-En-03504,Would be ever so grateful if you could record ...,False,False,False,False,True,False,False,False,False,False,False


In [ ]:
print(test_df['Tweet'][10])

Thanks to Dollis Hill's relentless and tenacious PC Leach and colleagues!


In [ ]:
def emotion(example):
  
  # print(example)
  encodings = tokenizer.encode_plus(
      example,
      None,
      add_special_tokens=True,
      max_length=MAX_LEN,
      padding='max_length',
      return_token_type_ids=True,
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt'
  )
  model.eval()
  with torch.no_grad():
      input_ids = encodings['input_ids'].to(device, dtype=torch.long)
      attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
      token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
      output = model(input_ids, attention_mask, token_type_ids)
      final_output = np.array(torch.sigmoid(output).cpu().detach().numpy().tolist())
      # print(final_output)
      # print(np.argwhere(final_output>0.7)[:,1])
      tl=np.array(target_list)
      print(tl[np.argwhere(final_output>0.7)[:,1]])
      

In [ ]:
emotion('Thanks to Dollis Hills relentless and tenacious PC Leach and colleagues!')

['joy' 'optimism']


In [ ]:
emotion('what a lovely day')

['joy']


In [ ]:
emotion('it was really stressful, i am afraid')

['fear' 'sadness']


In [ ]:
emotion('')

[]
